The next step is to make the RDF connecting cell types and markers. For that, I`ll use a different ontology/knowledge base which I (Tiago) mantain: https://celltypes.wiki.opencura.com/.

The batch edits will be done with Wikidata Integrator, as per this manual:

https://colab.research.google.com/drive/1KEvQaIOJyMMw0kSVC8TpNd8qhqoJ-e7B?usp=sharing


In [4]:
from wikidataintegrator import wdi_core, wdi_login
from wikidataintegrator.wdi_helpers import try_write
import os
from rdflib import Graph, URIRef
import pandas as pd
import pprint
from IPython.display import clear_output
from getpass import getpass


In [34]:
types = pd.read_csv("../results/cell_type_reference_from_panglao_to_wikidata_31_10_2020.csv")
types

,Unnamed: 0,panglao,wikidata
0,0,Basal cells,Q101062513
1,1,Trigeminal neurons,Q101062590
2,2,Juxtaglomerular cells,Q2596226
3,3,Pancreatic stellate cells,Q1164962
4,4,Fibroblasts,Q463418
...,...,...,...
184,184,Cortical interneurons,Q101004030
185,185,T cell naive,Q6959842
186,186,Peripheral blood mononuclear cells,Q736033
187,187,Paneth Cells,Q2004084


In [35]:
wikibase = "https://celltypes.wiki.opencura.com/w/api.php"
WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=wikibase)

In [36]:
for index, row in types.iterrows():
    print(row)
    statements = []
    #The item is a (P1) cell archetype (Q2)
    statements.append(wdi_core.WDItemID(prop_nr="P1", value="Q2"))

    # The item is a subclass (P2) of Cell of eukariotic organism (Q21)
    statements.append(wdi_core.WDItemID(prop_nr="P2", value="Q21"))


    # wikidata match (P8)

    wikidata_url = "http://www.wikidata.org/entity/" + row["wikidata"]

    statements.append(wdi_core.WDUrl(prop_nr="P8", value=wikidata_url))



    taxon_scope_value = "Q23"
    taxon_scope_property = "P5"
    statements.append(wdi_core.WDItemID(prop_nr=taxon_scope_property, value=taxon_scope_value))

    definition_value = 'Any cell identifiable as part of the "' + row["panglao"] + '" class by the English-speaking scientific community'
    definition_property = "P6"
    statements.append(wdi_core.WDString(prop_nr=definition_property, value=definition_value))


    wd_item = wdi_core.WDItemEngine(data=statements, mediawiki_api_url=wikibase)

    descriptions =   {
        "en": "cell archetype in mammals",
        "pt-br": "arquétipo celular de mamíferos"
    }
    
    wd_item.set_label(label=row["panglao"], lang="en")

    for lang, description in descriptions.items():
        wd_item.set_description(description, lang=lang)
    
    try:
        wd_item.write(login)
    except:
        print(row["panglao"] + " already on database")


ct
Error while writing to Wikidata
Mammary epithelial cells already on database
Unnamed: 0                 7
panglao       Parietal cells
wikidata            Q1640093
Name: 7, dtype: object
Error while writing to Wikidata
Parietal cells already on database
Unnamed: 0                           8
panglao       Peri-islet Schwann cells
wikidata                    Q101062584
Name: 8, dtype: object
Error while writing to Wikidata
Peri-islet Schwann cells already on database
Unnamed: 0                   9
panglao       Chromaffin cells
wikidata               Q919689
Name: 9, dtype: object
Error while writing to Wikidata
Chromaffin cells already on database
Unnamed: 0                 10
panglao       Dendritic cells
wikidata              Q506253
Name: 10, dtype: object
Error while writing to Wikidata
Dendritic cells already on database
Unnamed: 0            11
panglao       Tuft cells
wikidata       Q25325383
Name: 11, dtype: object
Error while writing to Wikidata
Tuft cells already on databa

TypeError: can only concatenate str (not "float") to str

The added cell archetypes are now available by query at https://tinyurl.com/y42c5lcw. As the names were used as labels, we have an 1-1 match. 


In [80]:
import pandas as pd 
from wikidata2df import wikidata2df

with open("../queries/ct_wikibase_archetypes.rq", "r") as f:
    query = f.read()



In [86]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://celltypes.wiki.opencura.com/query/sparql/")
sparql.setQuery(query)
sparql.setReturnFormat("JSON")
results = sparql.query().convert()


HTTPError: HTTP Error 308: Permanent Redirect

I failed to set the local query for the wikibase, but that is okay. 
Next steps:

- add strictu sensu cell types for humans
- add properties for markers (has markers, has_panglao_marker)
- add statements for markers

In [48]:
query

'PREFIX ct: <http://celltypes.wiki.opencura.com/entity/>\nPREFIX ctp: <http://celltypes.wiki.opencura.com/prop/direct/>\n\nselect ?item  ?itemLabel\nwhere {\n  ?item ctp:P1 ct:Q2.\n  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n\n  }'